**Sun Trail is a hiking improvement app**

In [ ]:
#imports

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pymongo

from apikey import key

**Sun Trail has access to  data from their followers on instragam**

In [134]:
#(calling on instagrams api to get the user information of all of suntrails followers)

url = 'https://randomuser.me/api/'
#Using a batch size of 50
userList = ['']*50

# *** ---------------------------------------------------- ***
# *** This cell must be executed before each batch attempt ***
# *** ---------------------------------------------------- ***

In [ ]:
#Make calls on randomuser api to generate user data

for i, u in enumerate(userList):
    r = requests.get(url).json()
    userList[i] = r['results'][0]
    

userList = list(filter(None, userList))

In [ ]:
userBank.append(userList)
len(userBank[3])
userBank[]

In [140]:
#Create our XL databank storing batches in a single list
BigBank = []

for ulist in userBank:
    for item in ulist:
        BigBank.append(item)

#Look at the size of that B--ank!
len(BigBank)

**What are you doing with all that user data SunTrail?**

"Well we are launching our latest and greatest ad campaign of course! The last one was a bit of a flop, but we are adamant that everyone loves to hike! We believe with some restructuring, and more of a targeted ad campaign we can reach our audience and get to market in no time"

**That sounds great**

"Yea"

**That user data seems a bit random to me...**

"Well we haven't gotten to the secrete sauce here mate. We're only going to be spending money on ads when a hike sounds appealing, like say it's below 60 F, nobody is interested. But say, 70 F or above?? Take a hike!

In [197]:
#Query weather api for current weather data in each users city
weatherList = []

for u in BigBank:
    city = u['location']['city']
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={key}'
    r = requests.get(url).json()
    weatherList.append(r)

In [167]:
#TEST CELL
#Find index values where openweathermap api has no data for corresponding city
#Create list of index values to drop
index_drop = []

for i, x in enumerate(weatherList):
    if weatherList[i]['cod']=='404':
        index_drop.append(i)
        


In [168]:
#Actualizing index pop values for processing
print(index_drop[:10])

for i, x in enumerate(index_drop):
    index_drop[i] = x-i
index_drop[:10]

[20, 28, 32, 39, 45, 78, 98, 108, 110, 119]


[20, 27, 30, 36, 41, 73, 92, 101, 102, 110]

In [170]:
#popping null data
for i in index_drop:
    weatherList.pop(i)
    BigBank.pop(i)

In [171]:
#Sanity Check (should be equally sized lists)

print(len(weatherList))
print(len(BigBank))

In [ ]:
#Re-Run the above ^ TEST CELL to endsure no remaining null values are in our data set

In [191]:
#DATA PREPARATION (grabbing and sorting relevant data features)

#Create lists with weather data to populate DataFrame columns
tempList = []
humList = []
visiList = []
windList = []
cloudList = []
zoneList = []

#Create lists with user data to populate DataFrame columns
genderList = []
fnameList = []
lnameList = []
cityList = []
stateList = []
countryList = []
emailList = []
usernameList = []
ageList = []

#Iterate through our data and sort into respective columns
for i, x in enumerate(weatherList):
    tempList.append(weatherList[i]['main']['temp'])
    humList.append(weatherList[i]['main']['humidity'])
    visiList.append(weatherList[i]['visibility'])
    windList.append(weatherList[i]['wind']['speed'])
    cloudList.append(weatherList[i]['clouds']['all'])
    zoneList.append(weatherList[i]['timezone'])
    
    genderList.append(BigBank[i]['gender'])
    fnameList.append(BigBank[i]['name']['first'])
    lnameList.append(BigBank[i]['name']['last'])
    cityList.append(BigBank[i]['location']['city'])
    stateList.append(BigBank[i]['location']['state'])
    countryList.append(BigBank[i]['location']['country'])
    emailList.append(BigBank[i]['email'])
    usernameList.append(BigBank[i]['login']['username'])
    ageList.append(BigBank[i]['dob']['age'])

In [195]:
#populate dataframe 

df = pd.DataFrame()
df.insert(0, 'Temperature', tempList)
df.insert(1, 'Humidity', humList)
df.insert(2, 'Visibility', visiList)
df.insert(3, 'Wind', windList)
df.insert(4, 'TimeZone', zoneList)
df.insert(5, 'Gender', genderList)
df.insert(6, 'First Name', fnameList)
df.insert(7, 'Last Name', lnameList)
df.insert(8, 'City', cityList)
df.insert(9, 'State', stateList)
df.insert(10, 'Country', countryList)
df.insert(11, 'Email', emailList)
df.insert(12, 'Username', usernameList)
df.insert(13, 'Age', ageList)

#Convert Kelvin temperature data to Fahrenheit
df['Temperature'] = 1.8*(df['Temperature'] - 273) + 32
df['Temperature'] = round(df['Temperature'], 2)

In [209]:
#Create a target group DataFrame where the temperature is 70 or greater, and wind is less than 5
targetDF = df.loc[df['Temperature']>69.99]
targetDF = targetDF.loc[targetDF['Wind']<5]

In [ ]:
#Convert Dataframes to dictionaries (preparing data for mongoDB)
#We are using a non-relational database 

#Store data in mongoDB database ('MongoDatabase database'? idk if im saying that right but anyways..)
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'TargetGroup' Collection in Mongo
db = client.TargetGroup
data = targetDF.to_dict(orient='records')
db.TargetGroup.insert_many(data)

In [234]:
# Define and populate 'SunFollowers' Collection in Mongo
db = client.SunFollowers
data = df.to_dict(orient='records')
db.SunFollowers.insert_many(data)

#Decided to insert 'df' into our database rather than 'BigBank' 
#because keeping the weather data on users we did not target could be valuable